In [90]:
import numpy as np
import matplotlib.pyplot as plt
from os.path import isfile, join
import pandas as pd
from scipy.signal import find_peaks


peak_frequency = []
num_peaks = []
run_num = []
num_files = 201 # num_files includes baseline!

for i in range(1, num_files, 2): # goes through odd files
    if i == 201:
        s = 'baseline'
    else:
        s = str(i) # changes int i into string s

    path1 = r'/Users/beecontreras/Desktop/2019_Duke_TUNL_REU/runs_x100_1/'
    path2 = '/LOGS/history.txt'
    file_path = path1 + s + path2

    with open( file_path, 'r') as f:

        info_starts = 0
        data = []
        all_models = []

        for line in f:

            info_starts += 1

            model_info = line.split(" ")    # splits the individual numbers in the lines

            model_info = np.array(list(filter(None, model_info)))[:-1:]
            
            if info_starts == 5: # identifies tau column
                last_column = line.split(" ")
                last_column = np.array(list(filter(None, last_column)))[-2]
                last_column = last_column.astype(int) -1
                
                
            if info_starts == 6:
            
                column_name = {}
                
                for k in range(last_column):
                    column_name[model_info[k]] = k
                    

            if info_starts >= 7:     # This is where we care about the data

                model_elements = model_info.astype(float)

                all_models.append(model_elements)

                num_models = len(all_models)

    # This creates lists for star age and log luminosity

    star_age = []
    log_lum = []
    row_num = []

    # Fills in the above lists with data

    for j in range(num_models):
        row_num.append(j)
        star_age.append(all_models[j][1])
        log_lum.append(all_models[j][column_name['log_L']])


    peaks_x = []
    peaks_y = []
    row_check = []
    x = []
    y = []
    av_lum = []
    total = 0

    # Filling lists with differences of luminosity points 
    
    for j in range(len(log_lum) - 1):     
            x.append(log_lum[j]-log_lum[j-1]) 
            y.append(log_lum[j+1]-log_lum[j]) 
            if log_lum[j] > 0 and log_lum[j+1] - log_lum[j] < .01:
                av_lum.append(log_lum[j])

    # Finds peaks by saying that if luminosities are increasing and then switch to decreasing at a luminosity point
    # then that point is a peak if it's above a certain threshold  
 
    for k in range(len(x)):
        if x[k] > 0 and y[k] < 0 and log_lum[k] > 4.5:
            row_check.append(row_num[k])
            peaks_x.append(star_age[k]*31536000) #converts years to seconds
            peaks_y.append(log_lum[k])
            
   # Checks for extra peaks included by seeing if the peak's row numbers are too close
    # (there's a peak approx. every 35 rows - I use 20 to be cautious)
    # if a peak is less than 20 rows away from the previous, I take the highest one as
    # the peak and discard the other since it is a data anomaly 
    
    row_check = np.array(list(row_check))
    offset = 0
    for m in range(len(row_check)-1):
        if (row_check[m+1] - row_check[m]) < 20:
            if peaks_y[m - offset] < peaks_y[(m-offset)+1]: 
                peaks_y.remove(peaks_y[(m-offset)])        
                peaks_x.remove(peaks_x[(m-offset)])
                offset += 1                                   
            else: 
                peaks_y.remove(peaks_y[(m-offset)+1]) 
                peaks_x.remove(peaks_x[(m-offset)+1])
                offset += 1
                
    # Finds Burst Frequency (if there's more than one peak)
    num_peaks.append(len(peaks_x))
    peak_dist_sum = 0

    if len(peaks_x) == 1:
        print('\n\n\n','Only one peak', '\n\n\n')
        run_num.append(s)
    else:
        for k in range(len(peaks_x)-1):

            peak_dist_sum += (peaks_x[k+1]- peaks_x[k])
            #gets sum distance between all peaks

        burst_frequency = peak_dist_sum / (len(peaks_x) - 1)
       # divides by number of peaks (minus one b/c its an avg. 
        # over distance B E T W E E N peaks)

        peak_frequency.append(burst_frequency)
        run_num.append(s)
        print(burst_frequency, '     run number: ',s, '\n\n')
        print(peaks_x, '\n\n',peaks_y, '\n\n')

peak_frequency = np.array(list(peak_frequency))

8647.752673095714      run number:  1 


[9978.194920603837, 18695.58090821429, 27308.319542103807, 35861.112787746424, 44506.732104092654, 53116.6982744136, 61732.80177848463, 70487.72659591345, 79129.12214472196, 87807.96897846527] 

 [4.912597846939045, 4.768875438731733, 4.750524291069779, 4.694121530105452, 4.657815911698427, 4.755889781750091, 4.697692258431629, 4.713419198273031, 4.762544218389054, 4.746545778931974] 


8642.796823936798      run number:  3 


[9978.195678372913, 18677.951217011752, 27397.290603707417, 36048.79674122675, 44703.47999872736, 53387.63814352791, 62083.69231724304, 70575.82738143495, 79121.97107206887, 87763.3670938041] 

 [4.9125233962433965, 4.791386411118668, 4.751436125661668, 4.751618134972885, 4.763694792175507, 4.768062787019031, 4.665952731536157, 4.598408562646297, 4.732619863530496, 4.7303242420399325] 


8636.635220643915      run number:  5 


[9978.19488481702, 18686.01588112924, 27275.970639761785, 35868.52352643668, 44577.223234092125,

8566.060308560254      run number:  49 


[9932.564131880905, 18563.50499043477, 27171.320931782026, 35767.06134151418, 44205.07188299285, 52795.777334879174, 61356.076820323884, 69956.59147236536, 78508.83399243721, 87027.10690892319] 

 [4.907203896110239, 4.779021853585716, 4.765207353837629, 4.736780769667158, 4.6834962003273946, 4.709660406708173, 4.732173426333274, 4.730975540858788, 4.644443500050706, 4.6950930391587615] 


8549.912879819989      run number:  51 


[9932.567143932112, 18546.924293165295, 27140.440904239436, 35691.430500695904, 44320.28446674477, 52853.643851305285, 61368.14836051494, 69982.2002228372, 78332.34449411849, 86881.78306231201] 

 [4.907109426283076, 4.77058035542563, 4.746502306170208, 4.712007706828956, 4.741836773473327, 4.690436642295974, 4.697178251545392, 4.681592930102621, 4.713040263247477, 4.6796942179068335] 


8552.611056112157      run number:  53 


[9932.178789585125, 18549.310798303257, 27084.402359579683, 35684.59694062431, 44228.23332

8598.78310734806      run number:  95 


[9932.567988721057, 18558.06125459233, 27180.914098612368, 35630.27041871028, 44311.243707900285, 52885.50796086093, 61476.60932841352, 70074.83880803092, 78767.32083464395, 87321.6159548536] 

 [4.906909675706645, 4.766104354814734, 4.74332745303714, 4.73766511113953, 4.691893325684808, 4.661682822398646, 4.6552532941504365, 4.749839653492048, 4.737229251478561, 4.755365585737091] 


8592.047958993036      run number:  97 


[9932.568522947793, 18601.8950079214, 27160.475292976575, 35784.07217778289, 44323.81073253803, 52892.808317912975] 

 [4.906941533127493, 4.7775778899078185, 4.746653924569029, 4.702598714323866, 4.7346863583143515, 4.720676677187383] 


8610.473071287703      run number:  99 


[9932.530578376503, 18621.92902018851, 27223.69130209075, 35767.383240746436, 44370.44711522434, 52984.89593481502] 

 [4.908379413348706, 4.773474561977358, 4.76100871209159, 4.759454482775609, 4.754169330761774, 4.704762235007017] 


8604.3522380

8561.208851564223      run number:  137 


[9932.559554882251, 18580.81809333999, 27156.37271358344, 35663.874531943875, 44225.72484138464, 52756.50447456426, 61370.67556195775, 69923.28006347084, 78458.46473520731, 86983.43921896025] 

 [4.907921881179018, 4.766320314101158, 4.7637269801490225, 4.719289584351359, 4.668660260490305, 4.755481769468331, 4.738847307309418, 4.739133303931336, 4.731859740004368, 4.602387325704265] 


8590.927321747928      run number:  139 


[9932.427797064342, 18585.038839487082, 27183.155547629158, 35737.882933337794, 44296.137084056056] 

 [4.909813014275795, 4.767284991594481, 4.754960678388116, 4.714744918508519, 4.689609618153979] 


8606.521648408021      run number:  141 


[9932.559926437692, 18565.981984281312, 27124.52896928396, 35772.03382404658, 44429.343896248996, 53033.63835517505, 61674.921265567704, 70227.30323898142, 78758.68708855378, 87391.25476210988] 

 [4.907776170898739, 4.753793314567401, 4.767185399590965, 4.735725887515934, 4.685

8570.824469767338      run number:  179 


[9932.56205766218, 18596.96318741829, 27210.7442641334, 35797.98785169337, 44357.87555501087, 52931.5479677846, 61436.0901293806, 70051.2046865462, 78561.94656733792, 87069.98228556823] 

 [4.907209470674504, 4.775659994051345, 4.7644003081297255, 4.706225922078288, 4.707874879271684, 4.693940076945901, 4.773572453330534, 4.740684935784543, 4.739253564524103, 4.742643720963678] 


8578.191991767082      run number:  181 


[9932.572934660173, 18605.163175818052, 27131.199785947374, 35728.84660506348, 44320.8963421065, 52815.728571887325, 61430.85228417687, 70008.19609546848, 78554.44765474804, 87136.3008605639] 

 [4.907182121911437, 4.773114588863226, 4.76757466719362, 4.71944492052174, 4.713802551059646, 4.650136908555666, 4.739269209300244, 4.73148132855443, 4.681615977192365, 4.733672221737131] 


8581.30326241045      run number:  183 


[9932.561759663799, 18597.51356686248, 27220.26679423003, 35772.415653492506, 44329.55515069578, 52802

In [91]:
# Finds %diff, fractional diff, and change in time of burst freq. from basline for every model
for i in range(len(peak_frequency) - 1):
    percent_diff = (abs(peak_frequency[-1] - peak_frequency[i])/ peak_frequency[-1])*100
    time_diff = abs(peak_frequency[-1] - peak_frequency[i])
    time_frac = peak_frequency[-1] / peak_frequency[i]
    
    print('% Diff.: ', percent_diff, '  Change in time: ', time_diff, '   Fraction: ',time_frac, '   Run: ', run_num[i], '\n\n')

% Diff.:  0.7323485373535643   Change in time:  62.871253509874805    Fraction:  0.9927297581364144    Run:  1 


% Diff.:  0.6746209006315298   Change in time:  57.91540435095885    Fraction:  0.9932989973580592    Run:  3 


% Diff.:  0.6028481760971415   Change in time:  51.753801058075624    Fraction:  0.9940076430535852    Run:  5 


% Diff.:  0.24138341013646933   Change in time:  20.72247952676844    Fraction:  1.0024196747948988    Run:  7 


% Diff.:  0.0940503123968988   Change in time:  8.074107794023803    Fraction:  1.0009413885027967    Run:  9 


% Diff.:  0.11423718593250748   Change in time:  9.807126949377562    Fraction:  1.0011436783653058    Run:  11 


% Diff.:  0.5279515917279807   Change in time:  45.3240181026631    Fraction:  0.9947482109864116    Run:  13 


% Diff.:  0.5629727219582792   Change in time:  48.33054060473296    Fraction:  0.9944017891802501    Run:  15 


% Diff.:  0.1955931890034355   Change in time:  16.791443340731348    Fraction:  1.0019597

In [1]:
y = np.linspace(1, 1)
df = pd.DataFrame(data = num_peaks, index = run_num)
# creates a data frame using pandas - essentially tells code below where the data is stored 

writer = pd.ExcelWriter('num_peaks_1.xlsx', engine='xlsxwriter')
df.to_excel(writer, index=True)
writer.save()
# The above creates, writes in the data, then saves and excel spreadsheet

NameError: name 'np' is not defined

In [94]:
print(num_peaks)

[10, 10, 10, 10, 10, 10, 10, 7, 9, 1, 10, 10, 1, 10, 10, 11, 10, 10, 1, 10, 10, 10, 5, 10, 10, 10, 7, 10, 11, 10, 3, 10, 10, 3, 10, 10, 9, 7, 10, 10, 10, 8, 10, 10, 5, 10, 10, 10, 6, 6, 10, 11, 10, 8, 11, 10, 10, 10, 10, 10, 10, 10, 6, 10, 10, 6, 10, 10, 10, 5, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 8, 10, 6, 11, 11, 1, 11, 10, 10, 10, 10, 10, 11, 10, 8, 10, 10, 10, 11]
